# 🌾 Agro Demand Forecasting - Exploratory Data Analysis

This notebook contains exploratory data analysis for the agro demand forecasting project.

## Objectives
- Understand data structure and quality
- Identify patterns and correlations
- Visualize key trends
- Generate insights for feature engineering

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Add src to path
import sys
import os
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

In [ ]:
# Import project modules
from src.data.scrapers import MockDataGenerator, MockWeatherGenerator
from src.data.pipeline import DataCleaner, FeatureEngineer
from src.data.database import db_manager

## 1. Data Loading and Initial Exploration

In [ ]:
# Generate sample data for exploration
print("Generating sample data...")

# Product data
mock_generator = MockDataGenerator()
products_df = mock_generator.generate_historical_data(days=365)

# Weather data
weather_generator = MockWeatherGenerator()
weather_df = weather_generator.generate_historical_weather(days=365)

print(f"Products data shape: {products_df.shape}")
print(f"Weather data shape: {weather_df.shape}")

In [ ]:
# Display basic information
print("=== PRODUCTS DATA INFO ===")
products_df.info()

print("\n=== WEATHER DATA INFO ===")
weather_df.info()

In [ ]:
# Display sample data
print("=== PRODUCTS DATA SAMPLE ===")
display(products_df.head())

print("\n=== WEATHER DATA SAMPLE ===")
display(weather_df.head())

## 2. Data Quality Analysis

In [ ]:
# Check missing values
print("=== MISSING VALUES - PRODUCTS ===")
missing_products = products_df.isnull().sum()
print(missing_products[missing_products > 0])

print("\n=== MISSING VALUES - WEATHER ===")
missing_weather = weather_df.isnull().sum()
print(missing_weather[missing_weather > 0])

In [ ]:
# Check duplicates
print(f"Products duplicates: {products_df.duplicated().sum()}")
print(f"Weather duplicates: {weather_df.duplicated().sum()}")

In [ ]:
# Statistical summary
print("=== PRODUCTS DATA STATISTICS ===")
display(products_df.describe())

print("\n=== WEATHER DATA STATISTICS ===")
display(weather_df.describe())

## 3. Data Visualization

In [ ]:
# Product categories distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Category distribution
category_counts = products_df['category'].value_counts()
axes[0, 0].pie(category_counts.values, labels=category_counts.index, autopct='%1.1f%%')
axes[0, 0].set_title('Product Categories Distribution')

# Price distribution
axes[0, 1].hist(products_df['current_price'], bins=30, alpha=0.7)
axes[0, 1].set_title('Price Distribution')
axes[0, 1].set_xlabel('Price (€)')
axes[0, 1].set_ylabel('Frequency')

# Promotion distribution
promo_counts = products_df['is_promotion'].value_counts()
axes[1, 0].bar(['No Promotion', 'Promotion'], promo_counts.values)
axes[1, 0].set_title('Promotion Distribution')
axes[1, 0].set_ylabel('Count')

# Availability distribution
avail_counts = products_df['is_available'].value_counts()
axes[1, 1].bar(['Not Available', 'Available'], avail_counts.values)
axes[1, 1].set_title('Availability Distribution')
axes[1, 1].set_ylabel('Count')

plt.tight_layout()
plt.show()

In [ ]:
# Time series analysis
# Convert date to datetime
products_df['date'] = pd.to_datetime(products_df['scraped_at']).dt.date
weather_df['date'] = pd.to_datetime(weather_df['date']).dt.date

# Aggregate by date
daily_products = products_df.groupby('date').agg({
    'current_price': 'mean',
    'is_promotion': 'mean',
    'is_available': 'mean',
    'promotion_percentage': 'mean'
}).reset_index()

daily_weather = weather_df.groupby('date').agg({
    'temperature_2m': 'mean',
    'precipitation': 'mean',
    'humidity': 'mean',
    'wind_speed': 'mean'
}).reset_index()

# Plot time series
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Price trend
axes[0, 0].plot(pd.to_datetime(daily_products['date']), daily_products['current_price'])
axes[0, 0].set_title('Average Price Trend')
axes[0, 0].set_ylabel('Price (€)')
axes[0, 0].tick_params(axis='x', rotation=45)

# Temperature trend
axes[0, 1].plot(pd.to_datetime(daily_weather['date']), daily_weather['temperature_2m'], color='red')
axes[0, 1].set_title('Temperature Trend')
axes[0, 1].set_ylabel('Temperature (°C)')
axes[0, 1].tick_params(axis='x', rotation=45)

# Promotion rate
axes[1, 0].plot(pd.to_datetime(daily_products['date']), daily_products['is_promotion'], color='green')
axes[1, 0].set_title('Promotion Rate')
axes[1, 0].set_ylabel('Promotion Rate')
axes[1, 0].tick_params(axis='x', rotation=45)

# Availability rate
axes[1, 1].plot(pd.to_datetime(daily_products['date']), daily_products['is_available'], color='orange')
axes[1, 1].set_title('Availability Rate')
axes[1, 1].set_ylabel('Availability Rate')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Weather patterns
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Temperature distribution
axes[0, 0].hist(weather_df['temperature_2m'], bins=30, alpha=0.7)
axes[0, 0].set_title('Temperature Distribution')
axes[0, 0].set_xlabel('Temperature (°C)')
axes[0, 0].set_ylabel('Frequency')

# Precipitation distribution
axes[0, 1].hist(weather_df['precipitation'], bins=30, alpha=0.7, color='blue')
axes[0, 1].set_title('Precipitation Distribution')
axes[0, 1].set_xlabel('Precipitation (mm)')
axes[0, 1].set_ylabel('Frequency')

# Seasonal temperature
season_temp = weather_df.groupby('season')['temperature_2m'].mean()
axes[1, 0].bar(season_temp.index, season_temp.values)
axes[1, 0].set_title('Average Temperature by Season')
axes[1, 0].set_ylabel('Temperature (°C)')

# Weather conditions
weather_conditions = [
    weather_df['is_rainy'].sum(),
    weather_df['is_hot'].sum(),
    weather_df['is_cold'].sum()
]
condition_labels = ['Rainy Days', 'Hot Days', 'Cold Days']
axes[1, 1].bar(condition_labels, weather_conditions)
axes[1, 1].set_title('Weather Conditions Count')
axes[1, 1].set_ylabel('Number of Days')

plt.tight_layout()
plt.show()

## 4. Correlation Analysis

In [ ]:
# Merge data for correlation analysis
merged_df = pd.merge(daily_products, daily_weather, on='date', how='inner')

# Select numeric columns for correlation
numeric_cols = [
    'current_price', 'is_promotion', 'is_available', 'promotion_percentage',
    'temperature_2m', 'precipitation', 'humidity', 'wind_speed'
]

correlation_matrix = merged_df[numeric_cols].corr()

# Plot correlation heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.2f')
plt.title('Correlation Matrix')
plt.tight_layout()
plt.show()

In [ ]:
# Scatter plots for key relationships
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Price vs Temperature
axes[0, 0].scatter(merged_df['temperature_2m'], merged_df['current_price'], alpha=0.6)
axes[0, 0].set_title('Price vs Temperature')
axes[0, 0].set_xlabel('Temperature (°C)')
axes[0, 0].set_ylabel('Price (€)')

# Promotion vs Price
axes[0, 1].scatter(merged_df['is_promotion'], merged_df['current_price'], alpha=0.6)
axes[0, 1].set_title('Promotion Rate vs Price')
axes[0, 1].set_xlabel('Promotion Rate')
axes[0, 1].set_ylabel('Price (€)')

# Availability vs Temperature
axes[1, 0].scatter(merged_df['temperature_2m'], merged_df['is_available'], alpha=0.6)
axes[1, 0].set_title('Availability vs Temperature')
axes[1, 0].set_xlabel('Temperature (°C)')
axes[1, 0].set_ylabel('Availability Rate')

# Precipitation vs Availability
axes[1, 1].scatter(merged_df['precipitation'], merged_df['is_available'], alpha=0.6)
axes[1, 1].set_title('Precipitation vs Availability')
axes[1, 1].set_xlabel('Precipitation (mm)')
axes[1, 1].set_ylabel('Availability Rate')

plt.tight_layout()
plt.show()

## 5. Feature Engineering Insights

In [ ]:
# Apply feature engineering pipeline
print("Applying feature engineering...")

# Clean data first
cleaner = DataCleaner()
products_clean = cleaner.clean_product_data(products_df)
weather_clean = cleaner.clean_weather_data(weather_df)

# Store in database and get merged data
db_manager.insert_products(products_clean)
db_manager.insert_weather(weather_clean)
merged_data = db_manager.get_merged_data()

# Apply feature engineering
feature_engineer = FeatureEngineer()
features_df = feature_engineer.create_features(merged_data, 'availability_rate')

print(f"Features dataset shape: {features_df.shape}")
print(f"Number of features created: {len(features_df.columns)}")

In [ ]:
# Display feature importance insights
print("=== FEATURE ENGINEERING INSIGHTS ===")
print(f"\nTotal features created: {len(features_df.columns)}")
print(f"\nFeature types:")
print(features_df.dtypes.value_counts())

# Show some engineered features
engineered_features = [col for col in features_df.columns if col.endswith(('_lag_1', '_rolling_mean_7', '_sin', '_cos'))]
print(f"\nSample engineered features ({len(engineered_features)} total):")
for feature in engineered_features[:10]:
    print(f"  - {feature}")

In [ ]:
# Visualize some key engineered features
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Rolling mean
if 'availability_rate_rolling_mean_7' in features_df.columns:
    axes[0, 0].plot(features_df['availability_rate_rolling_mean_7'][:100])
    axes[0, 0].set_title('7-Day Rolling Mean of Availability Rate')
    axes[0, 0].set_ylabel('Rate')

# Lag feature
if 'availability_rate_lag_1' in features_df.columns:
    axes[0, 1].plot(features_df['availability_rate_lag_1'][:100])
    axes[0, 1].set_title('1-Day Lag of Availability Rate')
    axes[0, 1].set_ylabel('Rate')

# Seasonal features
if 'month_sin' in features_df.columns:
    axes[1, 0].plot(features_df['month_sin'][:365])
    axes[1, 0].set_title('Monthly Seasonal Pattern (Sine)')
    axes[1, 0].set_ylabel('Value')

# Weather interaction
if 'temp_precip_interaction' in features_df.columns:
    axes[1, 1].plot(features_df['temp_precip_interaction'][:100])
    axes[1, 1].set_title('Temperature-Precipitation Interaction')
    axes[1, 1].set_ylabel('Interaction Value')

plt.tight_layout()
plt.show()

## 6. Key Insights Summary

In [ ]:
# Generate insights summary
print("=" * 60)
print("🌾 AGRO DEMAND FORECASTING - EXPLORATORY INSIGHTS")
print("=" * 60)

print("\n📊 DATA QUALITY:")
print(f"  • Total product records: {len(products_df):,}")
print(f"  • Total weather records: {len(weather_df):,}")
print(f"  • Date range: {products_df['scraped_at'].min()} to {products_df['scraped_at'].max()}")
print(f"  • Product categories: {products_df['category'].nunique()}")
print(f"  • Missing values: {products_df.isnull().sum().sum()}")

print("\n💰 PRICE ANALYSIS:")
print(f"  • Average price: €{products_df['current_price'].mean():.2f}")
print(f"  • Price range: €{products_df['current_price'].min():.2f} - €{products_df['current_price'].max():.2f}")
print(f"  • Promotion rate: {products_df['is_promotion'].mean():.2%}")
print(f"  • Average discount: {products_df[products_df['is_promotion']]['promotion_percentage'].mean():.1f}%")

print("\n🌤️ WEATHER PATTERNS:")
print(f"  • Average temperature: {weather_df['temperature_2m'].mean():.1f}°C")
print(f"  • Temperature range: {weather_df['temperature_2m'].min():.1f}°C to {weather_df['temperature_2m'].max():.1f}°C")
print(f"  • Total precipitation: {weather_df['precipitation'].sum():.1f}mm")
print(f"  • Rainy days: {weather_df['is_rainy'].sum()} ({weather_df['is_rainy'].mean():.1%})")

print("\n🔗 CORRELATIONS:")
price_temp_corr = correlation_matrix.loc['current_price', 'temperature_2m']
price_promo_corr = correlation_matrix.loc['current_price', 'is_promotion']
avail_temp_corr = correlation_matrix.loc['is_available', 'temperature_2m']

print(f"  • Price vs Temperature: {price_temp_corr:.3f}")
print(f"  • Price vs Promotion: {price_promo_corr:.3f}")
print(f"  • Availability vs Temperature: {avail_temp_corr:.3f}")

print("\n⚙️ FEATURE ENGINEERING:")
print(f"  • Total features created: {len(features_df.columns)}")
print(f"  • Time-based features: {len([col for col in features_df.columns if 'month' in col or 'day' in col])}")
print(f"  • Lag features: {len([col for col in features_df.columns if 'lag_' in col])}")
print(f"  • Rolling features: {len([col for col in features_df.columns if 'rolling_' in col])}")
print(f"  • Weather interactions: {len([col for col in features_df.columns if 'temp_' in col or 'precip_' in col])}")

print("\n💡 KEY INSIGHTS:")
print("  1. Strong seasonal patterns in both prices and weather")
print("  2. Promotions show negative correlation with prices (as expected)")
print("  3. Weather conditions impact product availability")
print("  4. Feature engineering creates rich temporal patterns")
print("  5. Data quality is good with minimal missing values")

print("\n🎯 RECOMMENDATIONS FOR MODELING:")
print("  • Use seasonal features (month_sin, month_cos)")
print("  • Include lag features for temporal dependencies")
print("  • Leverage weather interactions for external factors")
print("  • Consider promotion impact as key predictor")
print("  • Use rolling statistics for trend analysis")

print("=" * 60)